# Setup Ambiente di Lavoro

Prima di tutto, è **necessario** configurare l'ambiente di lavoro. La cella seguente clonerà il repository GitHub contenente tutti i moduli Python (`.py`) e si posizionerà nella directory corretta. 

**Esegui questa cella come primo passo.**

In [ ]:
# Clona il repository e posizionati nella directory corretta
# NOTA: Assicurati che l'URL del repository e il nome del branch siano corretti.
!git clone https://github.com/devedale/snn-ids.git snn-ids
%cd snn-ids
# Assicurati di usare il nome del branch corretto.
!git checkout feat/modular-ml-pipeline

# Manoscritto della Pipeline ML per Cybersecurity

Questo notebook è una guida interattiva e un manoscritto completo per una pipeline di machine learning **avanzata** per l'analisi di traffico di rete. È stato progettato con tre principi chiave in mente:

- **Modularità**: Ogni fase logica (preprocessing, training, predizione) è isolata nel proprio modulo Python (`.py`).
- **Configurabilità**: Tutta la pipeline è controllata da un singolo file, `config.py`.
- **Estensibilità**: La struttura è pensata per essere facilmente adattabile a nuovi dataset o a nuove architetture di modelli.

## 1. Architettura e Scelte Tecniche

La pipeline è composta dai seguenti elementi:

- `create_synthetic_data.py`: Uno script per generare dati di esempio realistici.
- `config.py`: Il cuore della configurazione. Qui si definiscono i percorsi, le colonne da usare e gli iperparametri per il training.
- `preprocessing/process.py`: Contiene tutta la logica per caricare i dati e trasformarli in un formato numerico che il modello può comprendere.
- `training/train.py`: Gestisce la creazione del modello, la grid search e il salvataggio del modello più performante.
- `prediction/predict.py`: Carica un modello salvato e lo usa per fare predizioni su nuovi dati.

## 2. Setup Iniziale

Installiamo le librerie necessarie.

In [ ]:
# Installa le dipendenze
!pip install pandas scikit-learn tensorflow faker

## 3. Guida Approfondita al File di Configurazione (`config.py`)

Questa sezione è la più importante per personalizzare la pipeline. Vediamo ogni parametro in dettaglio.

### `DATA_CONFIG`
Controlla quali dati vengono usati.
- `dataset_path`: Il percorso del tuo file CSV.
- `timestamp_column`: Il nome esatto della colonna che contiene il timestamp. Essenziale per le finestre temporali.
- `feature_columns`: La lista delle colonne che vuoi usare come feature iniziali. La pipeline si occuperà di trasformarle (es. one-hot encoding).
- `ip_columns_to_anonymize`: Lista delle colonne che contengono indirizzi IP. Verranno trasformate in ID numerici.
- `target_column`: Il nome della colonna che contiene l'etichetta da predire.

### `PREPROCESSING_CONFIG`
Controlla come i dati vengono trasformati in sequenze.
- `use_time_windows`: Se `True`, attiva la logica di finestre temporali. Se `False`, ogni riga è un campione a sé. Impostalo a `False` se vuoi usare un modello non sequenziale come `'dense'`.
- `window_size`: **Parametro cruciale**. Definisce quanti eventi (righe) compongono una singola sequenza. Una finestra più grande cattura pattern a lungo termine, ma richiede più memoria e dati. Una finestra piccola cattura pattern a breve termine. La scelta dipende dal tipo di attacco che vuoi rilevare.
- `step`: Di quanti eventi si sposta la finestra. Se `step = 1`, generi il massimo numero di finestre sovrapposte. Se `step = window_size`, le finestre non si sovrappongono. Un valore intermedio (es. `step = window_size / 2`) è un buon compromesso tra quantità di dati generati e ridondanza.

### `TRAINING_CONFIG`
Il cuore del training.
- `validation_strategy`: Come detto, `'k_fold'` è più robusto, `'train_test_split'` è più veloce.
- `model_type`: Scegli l'architettura. `'lstm'` è fatta per le sequenze create da `use_time_windows=True`.
- `hyperparameters`: **Qui avviene la magia della multi-permutazione**. Per ogni parametro, puoi inserire una lista di valori da testare. Lo script di training creerà un modello per *ogni singola combinazione possibile*.
  - **Esempio di Test Multi-Permutazione**: Se imposti:
    ```python
    "batch_size": [16, 32],
    "learning_rate": [0.001, 0.01],
    ```
    La grid search testerà 4 combinazioni: (16, 0.001), (16, 0.01), (32, 0.001), (32, 0.01).
  - **Consigli per il Tuning**:
    - `batch_size`: Parti con 32 o 64. Se il tuo dataset è enorme, prova valori più alti. Se il modello non converge bene, prova valori più bassi.
    - `epochs`: Parti con un numero medio (es. 20-50). Se vedi che il modello migliora ancora alla fine, aumenta le epoche.
    - `learning_rate`: È il parametro più sensibile. Un range comune da testare è `[0.01, 0.001, 0.0001]`.
    - `lstm_units`: Controlla la capacità del modello LSTM. Parti con valori come 32, 64 o 128. Aumenta se sospetti che il modello sia troppo semplice (underfitting), diminuisci se va in overfitting.

## 4. Esecuzione della Pipeline Completa
Ora che abbiamo capito la configurazione, eseguiamo l'intera pipeline.

In [ ]:
print("--- FASE 1: Generazione Dati ---")
from create_synthetic_data import generate_synthetic_data
generate_synthetic_data()

print("\n--- FASE 2: Preprocessing in Finestre Temporali ---")
from preprocessing.process import preprocess_data
X, y = preprocess_data()
if X is not None:
    print(f"\nShape di X: {X.shape}")
    print(f"Shape di y: {y.shape}")

print("\n--- FASE 3: Training del Modello ---")
from training.train import train_and_evaluate
_, best_model_path = train_and_evaluate()

print("\n--- FASE 4: Predizione su Nuovi Dati ---")
from prediction.predict import predict_on_window
from config import PREPROCESSING_CONFIG
window_size = PREPROCESSING_CONFIG.get('window_size', 10)
sample_window = [
    {
        "ip_sorgente": "192.168.1.10", "ip_destinazione": "10.0.0.5",
        "porta_sorgente": 12345, "porta_destinazione": 443,
        "protocollo": "UDP", "byte_inviati": 250, "byte_ricevuti": 1800
    } for _ in range(window_size)
]
prediction_label = predict_on_window(sample_window)
if prediction_label:
    print(f"\nRISULTATO FINALE: La finestra di dati è stata classificata come: '{prediction_label}'")

## 5. Guida Passo-Passo: Adattare la Pipeline al Tuo Dataset

Questa guida ti mostra come usare i tuoi dati.

### Passaggio 1: Aggiorna `config.py`

Immagina di avere un nuovo dataset `my_traffic.csv` con le seguenti colonne: `['Timestamp', 'SourceIP', 'DestIP', 'SourcePort', 'DestPort', 'ProtocolType', 'PacketCount', 'TrafficType']`.

Per adattare la pipeline, dovrai modificare `DATA_CONFIG` in `config.py` così:

```python
DATA_CONFIG = {
    "dataset_path": "data/my_traffic.csv",
    "timestamp_column": "Timestamp",
    "feature_columns": ["SourcePort", "DestPort", "ProtocolType", "PacketCount"],
    "ip_columns_to_anonymize": ["SourceIP", "DestIP"],
    "target_column": "TrafficType",
}
```
**Fatto!** La pipeline ora userà i tuoi dati.

### Passaggio 2: Creare Nuove Feature (Feature Engineering)

Spesso, le feature migliori non sono quelle originali, ma quelle che creiamo noi. Immaginiamo di voler creare una feature `is_well_known_port` che è `1` se la porta di destinazione è una porta comune (es. 80, 443) e `0` altrimenti.

1. Apri `preprocessing/process.py`.
2. Trova la sezione dove viene caricato il DataFrame `df`, subito dopo `pd.read_csv`.
3. **Aggiungi il tuo codice** per creare la nuova colonna:

```python
# Esempio da aggiungere in process.py
well_known_ports = [80, 443, 22, 21, 53]
df['is_well_known_port'] = df[DATA_CONFIG['feature_columns'][1]].isin(well_known_ports).astype(int)
```

4. Infine, **aggiorna `config.py`** per includere la tua nuova feature nella lista `feature_columns`:

```python
"feature_columns": ["SourcePort", "DestPort", "ProtocolType", "PacketCount", "is_well_known_port"],
```
Questo approccio ti dà la flessibilità di creare logiche di preprocessing complesse mantenendo la configurazione semplice e centralizzata.